## Seminar 1: Fun with Word Embeddings (3 points)

Today we gonna play with word embeddings: train our own little embedding, load one from   gensim model zoo and use it to visualize text corpora.

This whole thing is gonna happen on top of embedding dataset.

__Requirements:__  `pip install --upgrade nltk gensim bokeh` , but only if you're running locally.

In [7]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

zsh:1: no matches found: https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1


In [8]:
import numpy as np

data = list(open("./quora.txt", encoding="utf-8"))
data[50]

"What TV shows or books help you read people's body language?\n"

__Tokenization:__ типичным первым шагом в задаче nlp является разбиение исходных данных на слова.
Текст, с которым мы работаем, представлен в сыром формате: со всеми знаками препинания и смайлами, прикрепленными к некоторым словам, поэтому простым str.split не обойтись.

Let's use __`nltk`__ - a library that handles many nlp tasks like tokenization, stemming or part-of-speech tagging.

In [12]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [20]:
data_tok = []
for line in data:
    line = line.lower()
    tokens = tokenizer.tokenize(line)
    data_tok.append(tokens)

In [21]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

In [22]:
print([' '.join(row) for row in data_tok[:2]])

["can i get back with my ex even though she is pregnant with another guy ' s baby ?", 'what are some ways to overcome a fast food addiction ?']


__Word vectors:__ как говорится, существует более одного способа обучения вкраплений слов. Есть Word2Vec и GloVe с разными целевыми функциями. Затем есть fasttext, который использует модели на уровне символов для обучения вкраплений слов. 

Выбор огромен, поэтому давайте начнем с малого: __gensim__ - это еще одна библиотека nlp, которая содержит множество векторных моделей, в том числе и word2vec.

In [26]:
from gensim.models import Word2Vec
model = Word2Vec(data_tok, vector_size=32, min_count=5, window=5).wv  # define context as a 5-word window around the target word

In [30]:
# now you can get word vectors !
model.get_vector('anything')

array([-2.048604  ,  0.05068253,  0.6527221 ,  2.9996443 ,  2.1747391 ,
        1.943513  ,  3.1834464 , -2.7003179 ,  0.80580103,  3.4011054 ,
       -0.882452  ,  1.4105083 ,  4.531583  ,  0.6952587 ,  3.717811  ,
       -0.7748882 ,  0.10841208, -0.9013556 ,  0.53564346, -3.045519  ,
       -2.729716  , -1.159137  , -1.6058552 ,  0.44023302,  1.3961331 ,
       -1.1712145 , -0.7940845 , -0.652556  ,  0.57056934,  0.54742867,
       -0.49052277,  0.58534867], dtype=float32)

In [28]:
# or query similar words directly. Go play with it!
model.most_similar('bread')

[('rice', 0.9453203678131104),
 ('sauce', 0.927678108215332),
 ('butter', 0.9181720614433289),
 ('cheese', 0.9178261160850525),
 ('potato', 0.9043764472007751),
 ('beans', 0.9009194374084473),
 ('flour', 0.9004645943641663),
 ('fruit', 0.899115800857544),
 ('chocolate', 0.8969575762748718),
 ('soup', 0.893844723701477)]

### Using pre-trained model

Took it a while, huh? Now imagine training life-sized (100~300D) word embeddings on gigabytes of text: wikipedia articles or twitter posts. 

Thankfully, nowadays you can get a pre-trained word embedding model in 2 lines of code (no sms required, promise).

In [29]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

In [30]:
model.most_similar(positive=["coder", "analyst"], negative=["brain"])

[('consultant', 0.6947277784347534),
 ('programmer', 0.6648436188697815),
 ('developer', 0.6481666564941406),
 ('administrator', 0.6392731070518494),
 ('architect', 0.627932608127594),
 ('specialist', 0.6134839057922363),
 ('sharepoint', 0.603934109210968),
 ('merchandising', 0.6020367741584778),
 ('procurement', 0.6010934114456177),
 ('tibco', 0.5983521342277527)]

### Visualizing word vectors

One way to see if our vectors are any good is to plot them. Thing is, those vectors are in 30D+ space and we humans are more used to 2-3D.

Luckily, we machine learners know about __dimensionality reduction__ methods.

Let's use that to plot 1000 most frequent words

In [31]:
print(type(model))

<class 'gensim.models.keyedvectors.KeyedVectors'>


In [33]:
words = sorted(
    model.key_to_index.keys(),
    key=lambda word: model.get_vecattr(word, 'count'),
    reverse=True
)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [44]:
# for each word, compute it's vector with model
word_vectors = np.array([model[word] for word in words])


In [45]:
import numpy as np

assert isinstance(word_vectors, np.ndarray)
assert word_vectors.shape == (len(words), 100)
assert np.isfinite(word_vectors).all()

#### Linear projection: PCA

Простейшим методом линейного уменьшения размерности является __П__принципиальный __К__компонентный __А__анализ.

В геометрических терминах PCA пытается найти оси, вдоль которых происходит большая часть дисперсии. Если хотите, «естественные» оси.

<img src="https://github.com/yandexdataschool/Practical_RL/raw/master/yet_another_week/_resource/pca_fish.png" style="width:30%">


Она пытается разложить объектно-функциональную матрицу $X$ на две меньшие матрицы: $W$ и $\hat W$, минимизируя _среднюю квадратичную ошибку_:

$$\|(X W) \hat{W} - X\|^2_2 \to_{W, \hat{W}} \min$$
- $X \in \mathbb{R}^{n \times m}$ - object matrix (**centered**);
- $W \in \mathbb{R}^{m \times d}$ - matrix of direct transformation;
- $\hat{W} \in \mathbb{R}^{d \times m}$ - matrix of reverse transformation;
- $n$ samples, $m$ original dimensions and $d$ target dimensions;



In [49]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=2)
word_vectors_pca = pca.fit_transform(word_vectors)
scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_vectors_pca)

In [50]:
assert word_vectors_pca.shape == (len(word_vectors), 2), "there must be a 2d vector for each word"
assert max(abs(word_vectors_pca.mean(0))) < 1e-5, "points must be zero-centered"
assert max(abs(1.0 - word_vectors_pca.std(0))) < 1e-2, "points must have unit variance"

#### Let's draw it!

In [51]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [52]:
draw_vectors(word_vectors_pca[:, 0], word_vectors_pca[:, 1], token=words)

# hover a mouse over there and see if you can identify the clusters

figure(id='p1004', ...)

### Visualizing neighbors with t-SNE
PCA хорош, но он строго линеен и, следовательно, способен отразить только грубую высокоуровневую структуру данных.

Если мы хотим сосредоточиться на том, чтобы соседние точки были близки, мы можем использовать TSNE, который сам по себе является методом встраивания. Here you can read __[more on TSNE](https://distill.pub/2016/misread-tsne/)__.

In [55]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
word_tsne = tsne.fit_transform(word_vectors)
scaler = StandardScaler()
word_vectors_pca = scaler.fit_transform(word_tsne)

In [56]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

figure(id='p1106', ...)

### Visualizing phrases

Word embeddings can also be used to represent short phrases. The simplest way is to take __an average__ of vectors for all tokens in the phrase with some weights.

This trick is useful to identify what data are you working with: find if there are any outliers, clusters or other artefacts.

Let's try this new hammer on our data!


In [ ]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    vector = np.zeros([model.vector_size], dtype='float32')
    
    # YOUR CODE
    
    return vector
        
    

In [ ]:
vector = get_phrase_embedding("I'm very sure. This never happened to me before...")

assert np.allclose(vector[::10],
                   np.array([ 0.31807372, -0.02558171,  0.0933293 , -0.1002182 , -1.0278689 ,
                             -0.16621883,  0.05083408,  0.17989802,  1.3701859 ,  0.08655966],
                              dtype=np.float32))

In [ ]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = # YOUR CODE

In [ ]:
assert isinstance(phrase_vectors, np.ndarray) and np.isfinite(phrase_vectors).all()
assert phrase_vectors.shape == (len(chosen_phrases), model.vector_size)

In [ ]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE().fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - phrase_vectors_2d.mean(axis=0)) / phrase_vectors_2d.std(axis=0)

In [ ]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Finally, let's build a simple "similar question" engine with phrase embeddings we've built.

In [ ]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [ ]:
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argpartition, np.argsort
    """
    # YOUR CODE
    
    return <YOUR CODE: top-k lines starting from most similar>

In [ ]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

assert len(results) == 10 and isinstance(results[0], str)
assert results[0] == 'How do I get to the dark web?\n'
assert results[3] == 'What can I do to save the world?\n'

In [ ]:
find_nearest(query="How does Trump?", k=10)

In [ ]:
find_nearest(query="Why don't i ask a question myself?", k=10)

__Now what?__
* Try running TSNE on all data, not just 1000 phrases
* See what other embeddings are there in the model zoo: `gensim.downloader.info()`
* Take a look at [FastText](https://github.com/facebookresearch/fastText) embeddings
* Optimize find_nearest with locality-sensitive hashing: use [nearpy](https://github.com/pixelogik/NearPy) or `sklearn.neighbors`.